In [7]:
from datasets import load_dataset
import pandas as pd
ds = load_dataset("Sri1999/creditmix-dataset")
data_dup = pd.DataFrame(ds["train"])
data_dup.answer.value_counts()
data_dup

d:\anaconda\envs\unsloth_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,question,answer
0,"Age: 23.0, Occupation: Scientist, Annual Incom...",Good
1,"Age: 24.0, Occupation: Scientist, Annual Incom...",Good
2,"Age: 24.0, Occupation: Scientist, Annual Incom...",Good
3,"Age: 28.0, Occupation: Teacher, Annual Income:...",Good
4,"Age: 35.0, Occupation: Engineer, Annual Income...",Good
...,...,...
31863,"Age: 29.0, Occupation: Architect, Annual Incom...",Bad
31864,"Age: 29.0, Occupation: Architect, Annual Incom...",Bad
31865,"Age: 25.0, Occupation: Mechanic, Annual Income...",Good
31866,"Age: 25.0, Occupation: Mechanic, Annual Income...",Good


In [2]:
from datasets import Dataset
import random

def rebalance(ds):
    groups = {"Good": [], "Bad": [], "Standard": []}
    for row in ds:
        groups[row["answer"]].append(row)
    min_len = min(len(groups["Good"]), len(groups["Bad"]))
    balanced = groups["Good"][:min_len] + groups["Bad"][:min_len] + random.sample(groups["Standard"], min_len)
    random.shuffle(balanced)
    return Dataset.from_list(balanced)

ds_balanced = rebalance(ds["train"])
pd.DataFrame(ds_balanced).answer.value_counts()

answer
Standard    7588
Good        7588
Bad         7588
Name: count, dtype: int64

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-3B-Instruct",
    load_in_4bit=True,
    max_seq_length=max_seq_length,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W1009 23:52:38.903000 57184 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


d:\anaconda\envs\unsloth_env\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:341: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.9.9: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
SYSTEM_PROMPT = """
You are a financial risk analysis assistant.

Respond in the following format:
<reasoning>
(your reasoning here)
</reasoning>
<answer>
Choose exactly one of: "Good", "Bad", or "Standard"
</answer>
"""


# Convert to GSM8K-like format
ds = ds_balanced.map(lambda x: {
    "question": x["question"],
    "answer": x["answer"],
    "prompt": [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": x["question"]}
    ]
})
ds[0]


Map: 100%|██████████| 22764/22764 [00:01<00:00, 20872.73 examples/s]


{'question': 'Age: 16.0, Occupation: Lawyer, Annual Income: 92084.4, Outstanding Debt: 1265.51, Credit Utilization Ratio: 37.1719312819398, Payment Behaviour: High_spent_Small_value_payments',
 'answer': 'Standard',
 'prompt': [{'content': '\nYou are a financial risk analysis assistant.\n\nRespond in the following format:\n<reasoning>\n(your reasoning here)\n</reasoning>\n<answer>\nChoose exactly one of: "Good", "Bad", or "Standard"\n</answer>\n',
   'role': 'system'},
  {'content': 'Age: 16.0, Occupation: Lawyer, Annual Income: 92084.4, Outstanding Debt: 1265.51, Credit Utilization Ratio: 37.1719312819398, Payment Behaviour: High_spent_Small_value_payments',
   'role': 'user'}]}

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
lora_rank = 32


model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

Unsloth 2025.9.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [6]:
import re

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    # Extract model responses (what the LLM generated)
    responses = [completion[0]['content'] for completion in completions]

    # Get the question (last user message in prompt)
    q = prompts[0][-1]['content']

    # Extract only <answer>...</answer> from each response
    extracted_responses = [extract_xml_answer(r) for r in responses]

    # Debug print to see what's happening
    print(
        '-'*20,
        f"\nQuestion:\n{q}",
        f"\nGround Truth:\n{answer[0]}",
        f"\nModel Raw Response:\n{responses[0]}",
        f"\nExtracted Answer:\n{extracted_responses[0]}"
    )

    # Reward = 2.0 if correct label, else 0.0
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]


VALID_LABELS = {"Good", "Bad", "Standard"}

def cat_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]

    # Reward 0.5 if the model produces *any valid label*, else 0.0
    return [0.5 if r in VALID_LABELS else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]
def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [7]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",

    logging_steps=1,
    per_device_train_batch_size=6,   # <-- multiple of num_generations
    gradient_accumulation_steps=1,
    num_generations=6,               # <-- pairs with batch size above

    max_prompt_length=max_prompt_length,
    max_completion_length=max_seq_length - max_prompt_length,

    max_steps=100,
    save_steps=25,                   # frequent saves
    save_total_limit=2,              # keep space in outputs/
    max_grad_norm=0.1,
    report_to="none",
    output_dir="outputs",
)


In [8]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        cat_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = ds,
)


In [9]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 22,764 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 59,867,136 of 3,145,805,824 (1.90% trained)


-------------------- 
Question:
Age: 39.0, Occupation: Manager, Annual Income: 18683.75, Outstanding Debt: 3532.63, Credit Utilization Ratio: 28.682799619521195, Payment Behaviour: Low_spent_Small_value_payments 
Ground Truth:
Bad 
Model Raw Response:
<reasoning>
Based on the information provided, the individual has a credit utilization ratio of 28.68%, which is within a reasonable range since credit utilization generally should not exceed 20% to be considered good. The Payment Behaviour described as "Low_spent_Small_value_payments" suggests that the individual makes small and infrequent payments, which might indicate some instability but does not necessarily constitute a significant risk factor. However, given the specific value of 28.68%, the ratio is right on the borderline, which I would consider as Bad since it exceeds the usual safe upper limit. The other factors like age, occupation, and income do not significantly influence the credit utilization ratio directly but contribute i

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / xmlcount_reward_func / mean,rewards / xmlcount_reward_func / std,rewards / soft_format_reward_func / mean,rewards / soft_format_reward_func / std,rewards / strict_format_reward_func / mean,rewards / strict_format_reward_func / std,rewards / cat_reward_func / mean,rewards / cat_reward_func / std,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
1,-0.000000,1.339333,0.894065,206.666672,168.000000,250.000000,0.000000,206.666672,168.000000,250.000000,0.000000,-0.494000,0.199712,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,1.333333,1.032796
2,-0.000000,0.692667,1.078103,220.333344,115.000000,325.000000,0.000000,220.333344,115.000000,325.000000,0.000000,-0.474000,0.363922,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.666667,1.032796
3,0.000000,1.231833,1.033509,241.500000,158.000000,333.000000,0.000000,241.500000,158.000000,333.000000,0.004821,-0.601500,0.232967,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,1.333333,1.032796
4,0.000000,1.897500,0.449040,243.000000,160.000000,395.000000,0.000000,243.000000,160.000000,395.000000,0.007596,-0.602500,0.449040,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,2.000000,0.000000
5,0.000000,-0.024833,0.527939,217.833344,156.000000,439.000000,0.000000,217.833344,156.000000,439.000000,0.032073,-0.524833,0.527939,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000
6,0.000000,1.390334,1.185502,205.500000,141.000000,326.000000,0.000000,205.500000,141.000000,326.000000,0.032548,-0.443000,0.350355,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,1.333333,1.032796
7,0.000000,0.569833,0.917504,193.500000,143.000000,233.000000,0.000000,193.500000,143.000000,233.000000,0.007387,-0.263500,0.280618,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.333333,0.816497
8,0.000000,1.116000,1.161883,192.333344,127.000000,247.000000,0.000000,192.333344,127.000000,247.000000,0.012694,-0.384000,0.210648,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,1.000000,1.095445
9,0.000000,0.660333,0.904448,217.666672,129.000000,427.000000,0.000000,217.666672,129.000000,427.000000,0.013201,-0.506333,0.511885,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.666667,1.032796
10,0.000000,0.716167,0.893574,191.166672,135.000000,240.000000,0.000000,191.166672,135.000000,240.000000,0.017712,-0.450500,0.193410,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.666667,1.032796


Unsloth: Will smartly offload gradients to save VRAM!
-------------------- 
Question:
Age: 16.0, Occupation: Manager, Annual Income: 53451.48, Outstanding Debt: 4431.55, Credit Utilization Ratio: 26.73255968953977, Payment Behaviour: Low_spent_Small_value_payments 
Ground Truth:
Bad 
Model Raw Response:
<reasoning>
Given the information: the individual is 16 years old, which is below the common credit assessment age (typically considered to be 18 or older in many countries). This indicates that they might not have extensive credit history which is necessary for a detailed risk assessment. Additionally, they have a low credit score due to the described payment behavior (Low_spent_Small_value_payments), which suggests a history of making small or late payments. This could indicate financial difficulties or a lack of experience managing debts. The outstanding debt to income ratio of 8.11% (calculated as 4431.55 / 53451.48 * 100) is on the lower end, but still relatively high for being 16,

TrainOutput(global_step=100, training_loss=2.7697248988021172e-05, metrics={'train_runtime': 4940.7051, 'train_samples_per_second': 0.121, 'train_steps_per_second': 0.02, 'total_flos': 0.0, 'train_loss': 2.7697248988021172e-05})

In [ ]:
from unsloth import FastLanguageModel
from peft import PeftModel
import torch

SYSTEM_PROMPT = """
You are a financial risk analysis assistant.

Respond in the following format:
<reasoning>
(your reasoning here)
</reasoning>
<answer>
Choose exactly one of: "Good", "Bad", or "Standard"
</answer>
"""

# Load base + adapter for inference
base, tok = FastLanguageModel.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct", load_in_4bit=True, max_seq_length=1024,
)
model = PeftModel.from_pretrained(base, "outputs/checkpoint-100")  # inference mode
model.eval()
FastLanguageModel.for_inference(model)  # Unsloth's inference switch

# Use the same system prompt & template you trained with
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Age: 23.0, Occupation: Scientist, Annual Income: 19114.12, Outstanding Debt: 809.98, Credit Utilization Ratio: 35.03, Payment Behaviour: Low_spent_Small_value_payments"},
]
text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tok([text], return_tensors="pt").to(model.device)

with torch.inference_mode():
    out = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False,  # deterministic
        # no custom eos on a raw string; rely on template/EOS
    )

print(tok.decode(out[0], skip_special_tokens=True))


==((====))==  Unsloth 2025.9.9: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
system

You are a financial risk analysis assistant.

Respond in the following format:
<reasoning>
(your reasoning here)
</reasoning>
<answer>
Choose exactly one of: "Good", "Bad", or "Standard"
</answer>

user
Age: 23.0, Occupation: Scientist, Annual Income: 19114.12, Outstanding Debt: 809.98, Credit Utilization Ratio: 35.03, Payment Behaviour: Low_spent_Small_value_payments
assistant
<reasoning>
Based on the provided information, the individual's credit utilization ratio (35.03%) is within a reasonable range, w

In [11]:
from unsloth import FastLanguageModel
import os

model, tokenizer = FastLanguageModel.from_pretrained(
    r"D:\Narwal\fine-tuning-lora-qlora\outputs\checkpoint-100",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)


==((====))==  Unsloth 2025.9.9: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [12]:
model.push_to_hub(
    "Sri1999/Qwen2.5-3B-Instruct-CreditMix-QLORA",  
    tokenizer, 
    token = os.environ["HF_ACCESS_TOKEN"]
)

Saved model to https://huggingface.co/Sri1999/Qwen2.5-3B-Instruct-CreditMix-QLORA


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch, os

# === Paths ===
base_id   = "Qwen/Qwen2.5-3B-Instruct"                 # base model you trained from
lora_ckpt = r"D:\Narwal\fine-tuning-lora-qlora\outputs\checkpoint-100"  # your QLoRA checkpoint
out_dir   = r"D:\Narwal\fine-tuning-lora-qlora\merged-qwen2.5-3B"       # merged output

os.makedirs(out_dir, exist_ok=True)

# === Load tokenizer ===
tok = AutoTokenizer.from_pretrained(base_id, trust_remote_code=True)

# === Load base model in FP16 ===
model = AutoModelForCausalLM.from_pretrained(
    base_id,
    dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

# === Attach your QLoRA adapter ===
model = PeftModel.from_pretrained(model, lora_ckpt)

# === Merge the LoRA weights into the base model ===
model = model.merge_and_unload()

# === Save the merged model ===
model.save_pretrained(out_dir, safe_serialization=True)
tok.save_pretrained(out_dir)

print(f"✅ Merged model saved to: {out_dir}")


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.70s/it]


✅ Merged model saved to: D:\Narwal\fine-tuning-lora-qlora\merged-qwen2.5-3B


In [8]:
from llama_cpp import Llama

llm = Llama(
    model_path=r"D:\Narwal\fine-tuning-lora-qlora\qwen2.5-3b-f16-qlora.gguf",
    n_ctx=2048,
    n_threads=8,
    n_batch=512,
    verbose=False
)

def ask_financial_risk(question: str):
    prompt = f"""You are a financial risk analysis assistant.
Respond in the following format:
<reasoning>
(your reasoning here)
</reasoning>
<answer>
Choose exactly one of: "Good", "Bad", or "Standard"
</answer>

{question}
"""
    full = ""
    for chunk in llm.create_completion(prompt, max_tokens=256, stream=True):
        text = chunk["choices"][0]["text"]
        print(text, end="", flush=True)
        full += text
    return full

ask_financial_risk(
    "Age: 34.0, Occupation: Lawyer, Annual Income: 131313.4, Outstanding Debt: 352.16, Credit Utilization Ratio: 26.06663998426569, Payment Behaviour: Low_spent_Medium_value_payments"
)


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


<reasoning>
Based on the provided information, the individual is a 34-year-old lawyer with a moderate annual income of $131,313. This suggests they have a reasonable financial base. Their credit utilization ratio is 26.07%, which is considered good as it indicates they are using a small portion of their available credit. Their payment behavior is described as "Low_spent_Medium_value_payments," which is positive as it signifies responsible spending habits. However, the amount of outstanding debt ($352.16) is relatively low, which is favorable but not exceptional. Given these factors, the individual seems to manage their finances responsibly but not exceptionally well. 
</reasoning>
answer
Standard

'<reasoning>\nBased on the provided information, the individual is a 34-year-old lawyer with a moderate annual income of $131,313. This suggests they have a reasonable financial base. Their credit utilization ratio is 26.07%, which is considered good as it indicates they are using a small portion of their available credit. Their payment behavior is described as "Low_spent_Medium_value_payments," which is positive as it signifies responsible spending habits. However, the amount of outstanding debt ($352.16) is relatively low, which is favorable but not exceptional. Given these factors, the individual seems to manage their finances responsibly but not exceptionally well. \n</reasoning>\nanswer\nStandard'

In [17]:


# Test inference
prompt = """Age: 23.0, Occupation: Scientist, Annual Income: 19114.12,
Outstanding Debt: 809.98, Credit Utilization Ratio: 35.03,
Payment Behaviour: High_spent_Large_value_payments
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


RuntimeError: Expected all tensors to be on the same device, but got index is on cuda:0, different from other tensors on cpu (when checking argument in method wrapper_CUDA__index_select)

In [2]:
from llama_cpp import Llama

# Load your fine-tuned GGUF model
llm = Llama(
    model_path="qwen2.5-3b--lora-f16.gguf",
    n_ctx=4096,
    n_threads=6,
    n_gpu_layers=-1,  # -1 uses all available GPU layers
)
system_prompt = """You are a financial risk analysis assistant.
Respond in the following format:
<reasoning>
(your reasoning here)
</reasoning>
<answer>
Choose exactly one of: "Good", "Bad", or "Standard"
</answer>"""

# Your question (the actual input data)
question = """Age: 27.0, Occupation: Data Scientist, Annual Income: 72000.0,
Outstanding Debt: 2000.0, Credit Utilization Ratio: 18.7,
Payment Behaviour: High_spent_Large_value_payments"""

# Combine both into a single prompt — this is what the model sees
prompt = f"{system_prompt}\n\n{question}\n\nAnswer:"

# Run inference
output = llm(
    prompt=prompt,
    max_tokens=256,
    temperature=0.7,
    top_p=0.9,
    stop=["</answer>"],  # ensures it stops cleanly at your output tag
)

# Print model's answer
print(output["choices"][0]["text"])


llama_model_loader: loaded meta data with 26 key-value pairs and 434 tensors from qwen2.5-3b--lora-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Merged Qwen2.5 3B Lora
llama_model_loader: - kv   3:                           general.finetune str              = lora
llama_model_loader: - kv   4:                           general.basename str              = merged-qwen2.5
llama_model_loader: - kv   5:                         general.size_label str              = 3B
llama_model_loader: - kv   6:                          qwen2.block_count u32              = 36
llama_model_loader: - kv   7:                       qwen2.cont

KeyboardInterrupt: 